Импорт используемых библиотек

In [2]:
from Solve import solve_for_one_well_explicit
import matplotlib.pyplot as plt
import math as m
from well import Well
import numpy as np
import seaborn as sns
import time as t

Функция для выбора шага по пространству для равномерной сетки, чтобы все скважины попали в центры контрольных объемов

In [3]:
def choose_step(length, width, x_wells, y_wells):
    dx = dy = min(m.gcd(length, *x_wells), m.gcd(width, *y_wells))
    return dx, dy


Задание данных скважин: координаты по х и у, радиус ствола, постоянный дебит и номер

In [4]:
# данные для скважин скважины
wells = [
    Well(55, 25, 1.5, 0.01, 1),
    Well(70, 75, 1.5, 0.005, 2),
    Well(10, 75, 1.5, -0.005, 3),
    Well(55, 80, 1.5, -0.006, 4),
    
]

Ввод входных значений

In [5]:
# Ввод входных значений
length, width = 100, 100  # [м] геометрические размеры рассчитываемой области
dx, dy = choose_step(length, width, [well.x_w for well in wells],
                     [well.y_w for well in wells])  # [м] шаг по направлениям
Nx, Ny = int(length / dx) + 1, int(width / dy) + 1  # количество элементов
X = np.linspace(0, length, Nx)
Y = np.linspace(0, width, Ny)
T =  365 * 24 * 60 * 60 # время работы в секундах

B = 1.2  # Объемный коэффициент
h = 10  # толщина пласта

# параметры взяты плюс-минус от балды
viscosity = 10 * 10e-4  # Вязкость [Па * с]
compressibility = 5 * 10e-11  # [1/Па] сжимаемость
permeability = 100 * 10e-16  # [м2] проницаемость 
porosity = 0.05
eta = permeability/ (porosity * compressibility * viscosity)




Расчеты давлений

In [6]:
pressure_start = np.full((Nx, Ny), 0.0)
pressure_start[0, :] = 0
pressure_start[-1, :] = 0
pressure_start[:, 0] = 0
pressure_start[:, -1] = 0
pressure = pressure_start.copy()
permeability_matrix = np.full((Nx, Ny), permeability)
coef_matrix = B * viscosity / 2 / np.pi / permeability_matrix / h
eta_matrix = permeability_matrix / (porosity * compressibility * viscosity)
print(eta_matrix.mean())


0.4


In [7]:

begin = t.time()
for well in wells:
    # Используем явный метод
    well.pressure_field, well.pressure_well, well.time_well = solve_for_one_well_explicit(X.copy(), Y.copy(), well.x_w,
                                                                                          well.y_w, well.q, well.r_w,
                                                                                          coef_matrix, pressure_start.copy(),
                                                                                          T, eta_matrix)
    pressure += well.pressure_field
print(t.time() - begin)

C:\Users\Alexander\source\Studying\4 курс\Нефтегазовый инжениринг\Oil_and_Gas_Task1\Нефтегаз Ресчетка\Solve.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  max_dt = 1 / (2 * float(eta.mean()) * (dx ** 2 + dy ** 2)) * dx ** 2 * dy ** 2


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Визуализация

nvnn


In [10]:
fig, ax = plt.subplots(figsize=(8, 6), gridspec_kw={'hspace': 0})
for well in wells:
    ax.plot(well.time_well, well.pressure_well)
ax.set_xlabel('Время')
ax.set_title('Явный')
ax.set_ylabel('Давление на забоях скважин')
ax.legend([f' {well.number} скважина c дебитом {well.q} ' for well in wells])
plt.show()

#for style in plt.style.available == dark_background:
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.pcolormesh(X, Y, pressure, shading='auto', cmap='viridis')
cbar = plt.colorbar(cax, label='Давление')
ax.set_title(f'Распределение давления через {T/60/60/24} дней')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Явный')
plt.show()
print("nvnn")